In [4]:
%matplotlib inline
import torch, os, torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
import sys
sys.path.append('C:\\Users\\17374\\Anaconda3\\Lib\\site-packages')

In [6]:
DATA_ROOT = 'data'
all_labels_df = pd.read_csv(os.path.join(DATA_ROOT,'labels.csv'))
all_labels_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [10]:
breeds = all_labels_df.breed.unique()
breed2idx = dict((breed, idx) for idx, breed in enumerate(breeds))
idx2breed = dict((idx, breed) for idx, breed in enumerate(breeds))
len(breeds)

120

In [11]:
all_labels_df['label_idx'] = [breed2idx[b] for b in all_labels_df.breed]
all_labels_df.head()

,id,breed,label_idx
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,0
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,1
2,001cdf01b096e06d78e9e5112d419397,pekinese,2
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,3
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,4


In [54]:
class DogDataset(Dataset):
    def __init__(self, labels_df, img_path, transform = None):
        self.labels_df = labels_df
        self.img_path = img_path
        self.transform = transform
        
    def __len__(self):
        return self.labels_df.shape[0]
    
    def __getitem__(self, idx):
        image_name = os.path.join(self.img_path, self.labels_df.id[idx]) + '.jpg'
        img = Image.open(image_name)
        label = self.labels_df.label_idx[idx]
        
        if self.transform:
            img = self.transform(img)
        return img, label

In [64]:
IMG_SIZE = 224
BATCH_SIZE = 16
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]
CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if CUDA else "cpu")

In [65]:
train_transforms = transforms.Compose([transforms.Resize(IMG_SIZE),
                                      transforms.RandomResizedCrop(IMG_SIZE),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(30),
                                      transforms.ToTensor(),
                                      transforms.Normalize(IMG_MEAN, IMG_STD)])
val_transforms = transforms.Compose([transforms.Resize(IMG_SIZE),
                                    transforms.CenterCrop(IMG_SIZE),
                                    transforms.ToTensor(),
                                    transforms.Normalize(IMG_MEAN, IMG_STD)])

In [66]:
dataset_names = ['train', 'valid']
stratified_split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.1, random_state = 0)
train_split_idx, val_split_idx = next(iter(stratified_split.split(all_labels_df.id, all_labels_df.breed)))
train_df = all_labels_df.iloc[train_split_idx].reset_index()
val_df = all_labels_df.iloc[val_split_idx].reset_index()
print(len(train_df))
print(len(val_df))

9199
1023


In [67]:
image_transforms = {'train': train_transforms, 'valid':val_transforms}
train_dataset = DogDataset(train_df, os.path.join(DATA_ROOT, 'train'), transform = image_transforms['train'])
val_dataset = DogDataset(val_df, os.path.join(DATA_ROOT, 'train'), transform = image_transforms['valid'])
image_dataset = {'train': train_dataset, 'valid': val_dataset}
img_dataloader = {x: DataLoader(image_dataset[x], batch_size = BATCH_SIZE, shuffle = True, num_workers = 0) for x in dataset_names}
dataset_sizes = {x:len(image_dataset[x]) for x in dataset_names}

In [68]:
model_ft = models.resnet50(pretrained = True)
for param in model_ft.parameters():
    param.requires_grad = False
print(model_ft.fc)
num_fc_ftr = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_fc_ftr, len(breeds))
model_ft = model_ft.to(DEVICE)
print(model_ft)

Linear(in_features=2048, out_features=1000, bias=True)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
     

In [69]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([{'params': model_ft.fc.parameters()}],lr = 0.001)

In [70]:
def train(model, device, train_loader, epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        x,y = data
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {}\t Loss: {:.6f}'.format(epoch, loss.item()))

In [71]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            x,y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            test_loss += criterion(y_hat, y).item()
            pred = y_hat.max(1,keepdim = True)[1]
            correct += pred.eq(y.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss : {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(val_dataset), 100.*correct / len(val_dataset)))

In [72]:
for epoch in range(1,10):
    %time train(model = model_ft, device = DEVICE, train_loader = img_dataloader["train"], epoch = epoch)
    test(model = model_ft, device =DEVICE, test_loader = img_dataloader["valid"])

Train Epoch: 1	 Loss: 2.561230
Wall time: 1min 15s

Test set: Average loss : 0.0655, Accuracy: 733/1023 (72%)

Train Epoch: 2	 Loss: 2.619781
Wall time: 1min 19s

Test set: Average loss : 0.0549, Accuracy: 752/1023 (74%)

Train Epoch: 3	 Loss: 1.181528
Wall time: 1min 19s

Test set: Average loss : 0.0551, Accuracy: 759/1023 (74%)

Train Epoch: 4	 Loss: 2.806464
Wall time: 1min 20s

Test set: Average loss : 0.0573, Accuracy: 764/1023 (75%)

Train Epoch: 5	 Loss: 2.142424
Wall time: 1min 22s

Test set: Average loss : 0.0519, Accuracy: 767/1023 (75%)

Train Epoch: 6	 Loss: 2.197318
Wall time: 1min 22s

Test set: Average loss : 0.0536, Accuracy: 783/1023 (77%)

Train Epoch: 7	 Loss: 0.872373
Wall time: 1min 27s

Test set: Average loss : 0.0460, Accuracy: 798/1023 (78%)

Train Epoch: 8	 Loss: 1.781766
Wall time: 1min 26s

Test set: Average loss : 0.0505, Accuracy: 785/1023 (77%)

Train Epoch: 9	 Loss: 0.863043
Wall time: 1min 27s

Test set: Average loss : 0.0569, Accuracy: 772/1023 (75%)



In [73]:
in_list = []
def hook(module, input, output):
    for i in range(input[0].size(0)):
        in_list.append(input[0][i].cpu().numpy())

In [75]:
model_ft.avgpool.register_forward_hook(hook)

In [78]:
%%time
with torch.no_grad():
    for batch_idx, data in enumerate(img_dataloader["train"]):
        x,y = data
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        y_hat = model_ft(x)

Wall time: 2min


In [79]:
features = np.array(in_list)
np.save("features", features)

OSError: 923138048 requested and 0 written

In [80]:
features

array([[[[4.28585351e-01, 6.63867950e-01, 3.14915836e-01, ...,
          2.56059408e-01, 1.71858251e-01, 3.07308793e-01],
         [2.62154639e-01, 1.75246775e-01, 0.00000000e+00, ...,
          1.90203935e-01, 0.00000000e+00, 0.00000000e+00],
         [4.53138590e-01, 7.40848899e-01, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [1.38148713e+00, 9.37646329e-01, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [2.35656142e+00, 1.50223494e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [2.19017911e+00, 2.67038703e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

        [[5.99086434e-02, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [4.69466567e-01, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        